In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
from datetime import datetime

# 지역 이름과 코드 매핑
regions = {
    "남구": "5",
    "부산진구": "8",
    "해운대구": "18",
    "동래구": "7",
    "사하구": "12",
    "북구": "10",
    "사상구": "11"
}


target_date = "2023-04-13"

# 데이터 수집 함수
def fetch_region_data(region_name, region_code, date):
    url = "https://www.busan.go.kr/depart/abnecessity01"
    params = {
        "isMart": "true",
        "kind": "",
        "schDate": date,
        "schCode": "0",
        "schLocale": region_code,
        "schMart": "0"
    }

    response = requests.get(url, params=params)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table", class_="boardList")

    region_data = []

    if table:
        rows = table.find_all("tr")
        for row in rows:
            tds = row.find_all("td")
            td_texts = [td.get_text(strip=True) for td in tds]
            if len(td_texts) >= 5:
                item_name = td_texts[0]
                spec = td_texts[1]
                try:
                    lowest_price = int(td_texts[2].replace(",", ""))
                    highest_price = int(td_texts[3].replace(",", ""))
                except ValueError:
                    continue
                avg_price = td_texts[4]
                region_data.append([date, item_name, spec, lowest_price, highest_price, avg_price, region_name])
    return region_data

# 데이터 수집 실행
all_data = []
for region_name, region_code in regions.items():
    all_data.extend(fetch_region_data(region_name, region_code, target_date))

# DataFrame 생성
columns = ["날짜", "품목", "규격", "최저가", "최고가", "금주평균가", "지역"]
df = pd.DataFrame(all_data, columns=columns)

# 최저가, 최고가 각각 추출
min_df = df.loc[df.groupby("품목")["최저가"].idxmin()][["품목", "규격", "최저가", "지역", "금주평균가"]].reset_index(drop=True)
max_df = df.loc[df.groupby("품목")["최고가"].idxmax()][["품목", "최고가", "지역"]].reset_index(drop=True)

# 컬럼명 변경
min_df = min_df.rename(columns={"지역": "최저가_지역"})
max_df = max_df.rename(columns={"지역": "최고가_지역"})

# 병합 및 계산
merged_df = pd.merge(min_df, max_df, on="품목")
merged_df["가격차이"] = merged_df["최고가"] - merged_df["최저가"]
merged_df["차이비율(%)"] = (merged_df["가격차이"] / merged_df["최저가"] * 100).round(2).astype(str) + "%"

# 날짜 열 추가
merged_df["날짜"] = target_date

# 열 정리 및 출력
merged_df = merged_df[["날짜", "품목", "규격", "최저가", "최저가_지역", "최고가", "최고가_지역", "가격차이", "차이비율(%)", "금주평균가"]]
display(merged_df)

# 저장 (선택)
merged_df.to_csv("2023_total_2q.csv", index=False, encoding="utf-8-sig")


,날짜,품목,규격,최저가,최저가_지역,최고가,최고가_지역,가격차이,차이비율(%),금주평균가
0,2023-04-13,가루비누,"세탁용세제(LG테크),가루형,3.0kg",7416,해운대구,17250,해운대구,9834,132.61%,"11,197"
1,2023-04-13,간장,오복왕표 0.9ℓ 1병,5800,부산진구,8351,동래구,2551,43.98%,"5,800"
2,2023-04-13,갈치,60cm정도 500g 1마리(냉동),6464,해운대구,19040,동래구,12576,194.55%,"13,095"
3,2023-04-13,고등어,30㎝정도 500g 1마리,4500,동래구,11640,사상구,7140,158.67%,"4,700"
4,2023-04-13,달걀,오경슈퍼란60g정도 10개,2990,사하구,4400,동래구,1410,47.16%,"3,490"
5,2023-04-13,닭고기,육계 1.0㎏,5980,해운대구,14990,사상구,9010,150.67%,"8,299"
6,2023-04-13,대파,1.0㎏,2280,북구,4316,사하구,2036,89.3%,"2,885"
7,2023-04-13,돼지고기,삼겹살 500g,7900,해운대구,19450,부산진구,11550,146.2%,"11,700"
8,2023-04-13,두부,500g 판두부(국산포장두부 420g) 1모,1500,북구,5620,해운대구,4120,274.67%,"3,347"
9,2023-04-13,라면,신라면 120g 1봉지,820,남구,838,사하구,18,2.2%,820
